# Table of Contents
* [Introduction](#section0)
* [Data pre-processing](#section1)
* [Data exploratory](#section2)
* [Merging beer styles into supra-styles](#section3)
* [Numerical description of the supra-styles](#section4)
* [Textual description of the supra-styles](#section5)
* [Categorization of users](#section6)
* [Finding the favorite supra-styles for each category of users](#section7)
* [Finding the beers to put on the shelves](#section8)

# Imports

## Introduction <a class="anchor" id="section0"></a>

## Data pre-processing <a class="anchor" id="section1"></a>

For this project two dataset have been provided: data from [BeerAdvocate](https://www.beeradvocate.com) as well as from [RateBeer](https://www.ratebeer.com/) websites.
Both dataset are organized following the same relational model shown bellow.

### ER Diagram:
<img src="figures/ER_diagram.png" alt="ER_Diagram" width="800"/>

### Relations:
<img src="figures/relations.png" alt="relations" width="800"/>

The following analysis will be done on `BeerAdvocate` dataset. Raw data are available [here](https://drive.google.com/drive/folders/1Wz6D2FM25ydFw_-41I9uTwG9uNsN4TCF).
Processed data are available [here](https://drive.switch.ch/index.php/s/QBPV4ptiUoV8XER). In order to reproduce the analysis a data folder with the following architecture should be placed in the root directory of the repository:

<pre>  
├─── data
    ├─── raw
      ├─── BeerAdvocate
        ├─── beers.csv
        ├─── breweries.csv
        ├─── ratings.txt
        ├─── reviews.txt
        ├─── users.csv
      ├─── RateBeer
        ├─── beers.csv
        ├─── breweries.csv
        ├─── ratings.txt
        ├─── reviews.txt
        ├─── users.csv
    ├─── processed
      ├─── BeerAdvocate
        ├─── beers.pkl
        ├─── breweries.pkl
        ├─── ratings.pkl
        ├─── reviews.pkl
        ├─── users.pkl
      ├─── RateBeer
        ├─── beers.pkl
        ├─── breweries.pkl
        ├─── ratings.pkl
        ├─── reviews.pkl
        ├─── users.pkl 
</pre>

If one wants to manually re-produced the processed data from the raw data, simply move to the `src` folder and execute:
```
  python process_raw_data.py
```
Make sure to have all the requirements and that the data in the raw data folder have been uncompressed. Be aware that running the script took around 20 min on Apple silicon M1 Pro. 

## Data exploratory <a class="anchor" id="section2"></a>

## Merging beer styles into supra-styles <a class="anchor" id="section3"></a>

## Numerical description of the supra-styles <a class="anchor" id="section4"></a>

## Textual description of the supra-styles <a class="anchor" id="section5"></a>

## Categorization of users <a class="anchor" id="section6"></a>